In [1]:
# pfi 확인
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sub_df = pd.read_csv('./sample_submission.csv')

In [3]:
# 각 target 별로 분할하여 각 target의 확률을 예측

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost
from sklearn.ensemble import RandomForestClassifier

# 개발자가 만든 코드를 그대로 가져와 학습함
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier


def train_xgboost(target):
    xgb = xgboost.XGBClassifier()
    xgb.fit(X_train, target)
    return xgb

def prediction_xgboost(target, model):  
    
    y_pred = model.predict(target)
    y_sub = model.predict_proba(test_df)[:, 1]
    #print(f'{idx}', accuracy_score(y_test, y_pred))
    
    #y_pred = xgb.predict(test_df)
    #y_pred = pd.DataFrame(y_pred)
    return y_pred, y_sub

def train_random(target):
    rf = RandomForestClassifier()
    rf.fit(X_train, target)
    return rf

def prediction_rf(target, model):
    y_pred = model.predict(target)
    y_sub = model.predict_proba(test_df)[:, 1]
    return y_pred, y_sub

def train_voting(target, rf_model, xg_model):
    vt = VotingClassifier(estimators=[("Random Forest",rf_model),
                                               ("XGBoost",xg_model)],
                                  voting = 'soft')
    vt.fit(X_train, target)
    return vt

def prediction_vt(target, model):
    y_pred = model.predict(target)
    y_sub = model.predict_proba(test_df)[:, 1]
    
    # print(f'{idx}', accuracy_score(y_test, y_pred))
    
    #y_pred = rf.predict(test_df)
    #y_pred = pd.DataFrame(y_pred)
    return y_pred, y_sub
   

In [4]:
train_df.columns

Index(['id', 'X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum',
       'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
       'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
       'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
       'SigmoidOfAreas', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults'],
      dtype='object')

combine = {'train_df':train_df, 'test_df':test_df}

for idx in combine:
    typeofsteel = data[['TypeOfSteel_A300','TypeOfSteel_A400']]
    print(typeofsteel)
    steel = []
    for i in typeofsteel.index:
        if typeofsteel['TypeOfSteel_A300'][i] == 1:
            steel.append(0)
        elif typeofsteel['TypeOfSteel_A300'][i] == 0:
            steel.append(1)
    # print(f'{data}:', steel)
        #steel.drop([i], axis = 0, inplace=True)
    df = pd.DataFrame(steel)
    df.columns = ['TypeOfSteel']
    #print(df)
    data = pd.concat([data, df], axis = 1)
    data.drop(['TypeOfSteel_A300','TypeOfSteel_A400'], axis = 1, inplace=True )
    

In [5]:
# 제출에 사용할 id column
passid = test_df['id']
train_df.drop(['id'], axis = 1, inplace= True)
test_df.drop(['id'], axis = 1, inplace= True)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19219 entries, 0 to 19218
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   X_Minimum              19219 non-null  int64  
 1   X_Maximum              19219 non-null  int64  
 2   Y_Minimum              19219 non-null  int64  
 3   Y_Maximum              19219 non-null  int64  
 4   Pixels_Areas           19219 non-null  int64  
 5   X_Perimeter            19219 non-null  int64  
 6   Y_Perimeter            19219 non-null  int64  
 7   Sum_of_Luminosity      19219 non-null  int64  
 8   Minimum_of_Luminosity  19219 non-null  int64  
 9   Maximum_of_Luminosity  19219 non-null  int64  
 10  Length_of_Conveyer     19219 non-null  int64  
 11  TypeOfSteel_A300       19219 non-null  int64  
 12  TypeOfSteel_A400       19219 non-null  int64  
 13  Steel_Plate_Thickness  19219 non-null  int64  
 14  Edges_Index            19219 non-null  float64
 15  Em

## Preprocessing

In [7]:
# X_range = X_max - X_min
# Y_range = Y_max - Y_min

def maxmin(data):
    data['X_range'] = data['X_Maximum'] - data['X_Minimum']
    data['Y_range'] = data['Y_Maximum'] - data['Y_Minimum']
    data = data.drop(['X_Maximum','X_Minimum','Y_Maximum','Y_Minimum'], axis = 1)
    return data
# lumino_range = 'Maximum_of_Luminosity'-'Minimum_of_Luminosity'
def lumino_range(data):
    data['Luminosity_Range'] = data['Maximum_of_Luminosity'] - data['Minimum_of_Luminosity']
    data = data.drop(['Minimum_of_Luminosity','Maximum_of_Luminosity'], axis = 1)
    return data

def typeofsteel(data):
    steel_type = data[['TypeOfSteel_A300','TypeOfSteel_A400']]
    steel = []
    
    for i in range(len(steel_type)-1):
        if ((steel_type['TypeOfSteel_A300'][i] == 1) & (steel_type['TypeOfSteel_A400'][i]) == 1) == True:
            idx = i
        
        steel_type = data[['TypeOfSteel_A300','TypeOfSteel_A400']]
    # print(data)
    
    for i in range(len(steel_type)-1):

        if steel_type['TypeOfSteel_A300'][i] == 1:
            steel.append(0)
        elif steel_type['TypeOfSteel_A300'][i] == 0:
            steel.append(1)
        else:
            steel.append(2)
    
    # print(f'{data}:', steel)
        #steel.drop([i], axis = 0, inplace=True)
    df = pd.DataFrame(steel)
    df.columns = ['TypeOfSteel']
    df.reset_index()
    data.reset_index()
    #print(df)
    data = pd.concat([data, df], axis = 1)

    # 이상치 대체
    data['TypeOfSteel'].fillna(2, inplace = True)

    data.drop(['TypeOfSteel_A300','TypeOfSteel_A400'], axis = 1, inplace=True )
    
    return data

In [8]:
# feature engineering 
train_df = maxmin(train_df)
test_df = maxmin(test_df)

train_df = lumino_range(train_df)
test_df = lumino_range(test_df)

train_df = typeofsteel(train_df)
test_df = typeofsteel(test_df)

In [9]:
# target, feaure column 분리
test_col = test_df.columns
target_col = train_df.columns


for i in test_col:
    target_col = target_col.drop(i)
    
# target_feature 와 분리
target = train_df[target_col]
train_df = train_df.drop(target_col, axis = 1)

In [10]:
# standard scaling 을 위한 label  추출
feature = train_df.columns
feature = feature.to_list()

feature.remove('TypeOfSteel')

In [11]:
# standard scaling
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
std.fit(train_df[feature])
train_df[feature] = std.transform(train_df[feature])
test_df[feature] = std.transform(test_df[feature])

In [19]:
prep = pd.concat([train_df,target], axis =1)

In [32]:
prep.to_csv('./dev_preprocessing.csv', index= False)

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19219 entries, 0 to 19218
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Pixels_Areas           19219 non-null  float64
 1   X_Perimeter            19219 non-null  float64
 2   Y_Perimeter            19219 non-null  float64
 3   Sum_of_Luminosity      19219 non-null  float64
 4   Length_of_Conveyer     19219 non-null  float64
 5   Steel_Plate_Thickness  19219 non-null  float64
 6   Edges_Index            19219 non-null  float64
 7   Empty_Index            19219 non-null  float64
 8   Square_Index           19219 non-null  float64
 9   Outside_X_Index        19219 non-null  float64
 10  Edges_X_Index          19219 non-null  float64
 11  Edges_Y_Index          19219 non-null  float64
 12  Outside_Global_Index   19219 non-null  float64
 13  LogOfAreas             19219 non-null  float64
 14  Log_X_Index            19219 non-null  float64
 15  Lo

In [13]:
# correlation
corr_df = pd.concat([train_df, target], axis = 1)

corr_test = corr_df.corr()
corr_test_target = abs(corr_test[target.columns])
corr_test_target.drop(corr_test_target.loc[target.columns], axis =0, inplace=True)
# corr_target = abs(corr_test[target])
# 
#corr_target = abs(corr_test['Survived'])
#print(corr_target.sort_values(ascending = False))

In [58]:
corr_test_target

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
Pixels_Areas,0.101281,0.081555,0.700990,0.077462,0.053540,0.224011,0.195896
X_Perimeter,0.104311,0.070424,0.675054,0.084609,0.050180,0.218269,0.186208
Y_Perimeter,0.064957,0.062085,0.630285,0.100200,0.019329,0.227242,0.174562
Sum_of_Luminosity,0.093146,0.059585,0.640571,0.074237,0.046705,0.207291,0.184619
Length_of_Conveyer,0.158662,0.168229,0.272570,0.110411,0.052633,0.183830,0.109347
Steel_Plate_Thickness,0.080028,0.010618,0.303069,0.079759,0.011779,0.008440,0.202890
Edges_Index,0.019341,0.083029,0.357780,0.143549,0.065452,0.192625,0.078341
Empty_Index,0.133620,0.085459,0.204526,0.065597,0.011673,0.133633,0.005165
Square_Index,0.150471,0.005760,0.050909,0.078031,0.131697,0.175254,0.007310
Outside_X_Index,0.139169,0.087010,0.778807,0.091395,0.071054,0.232242,0.215440


In [63]:
# correlation
corr_df = pd.concat([train_df, target], axis = 1)

corr_test = corr_df.corr()
corr_test_target = abs(corr_test[target.columns])
corr_test_target.drop(corr_test_target.loc[target.columns], axis =0, inplace=True)

target_list = {}
for idx in target.columns:
    target_list[idx] = corr_test_target[idx].sort_values(ascending=False)
    print(f'================\n{idx}\n================\n',target_list[idx][:10])

Pastry
 Orientation_Index       0.261296
Outside_Global_Index    0.223759
Edges_Y_Index           0.201764
Log_X_Index             0.177292
Length_of_Conveyer      0.158662
Square_Index            0.150471
Outside_X_Index         0.139169
Empty_Index             0.133620
X_range                 0.112487
X_Perimeter             0.104311
Name: Pastry, dtype: float64
Z_Scratch
 TypeOfSteel           0.190117
Length_of_Conveyer    0.168229
Luminosity_Range      0.107375
Outside_X_Index       0.087010
Empty_Index           0.085459
Edges_Index           0.083029
Pixels_Areas          0.081555
X_range               0.077210
X_Perimeter           0.070424
Orientation_Index     0.069013
Name: Z_Scratch, dtype: float64
K_Scatch
 Log_X_Index          0.782062
Outside_X_Index      0.778807
LogOfAreas           0.745486
Luminosity_Range     0.701169
Pixels_Areas         0.700990
Edges_Y_Index        0.678424
X_Perimeter          0.675054
Sum_of_Luminosity    0.640571
X_range              0.633929


In [14]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# f1 스코어 저장
sub = pd.DataFrame()
xgb_model = {}
rf_model = {}
vt_model = {}

X_train, X_test, y_train, y_test = train_test_split(train_df, target, random_state= 42, test_size=0.2)

for target_name in target_col:
    xgb = train_xgboost(y_train[target_name])
    xgb_model[target_name] = xgb
    rf = train_random(y_train[target_name])
    rf_model[target_name] = rf
    vt = train_voting(y_train[target_name], xgb, rf)
    vt_model[target_name] = vt
    
    pred_xgb, y_xgb = prediction_xgboost(X_test,xgb)
    pred_rf, y_rf = prediction_rf(X_test,rf)
    pred_vt, y_vt = prediction_vt(X_test, vt)
    
    print(f'xgb_score_{target_name}:', accuracy_score(y_test[target_name],pred_xgb))
    print(f'rf_score_{target_name}:', accuracy_score(y_test[target_name],pred_rf))
    print(f'vt_score_{target_name}:', accuracy_score(y_test[target_name],pred_vt))
    
    # voting ensemble 사용을 위한 코드
    
    pred_vt_sub, y_vt_sub = prediction_vt(test_df, vt)
    #print(f'{target_name}:',type(y_vt_sub),'\n',y_vt_sub)
    sub[target_name] = y_vt_sub

xgb_score_Pastry: 0.9209157127991675
rf_score_Pastry: 0.923777315296566
vt_score_Pastry: 0.9250780437044746
xgb_score_Z_Scratch: 0.9508324661810614
rf_score_Z_Scratch: 0.9534339229968782
vt_score_Z_Scratch: 0.9513527575442248
xgb_score_K_Scatch: 0.964360041623309
rf_score_K_Scatch: 0.964360041623309
vt_score_K_Scatch: 0.9646201873048907
xgb_score_Stains: 0.9830905306971904
rf_score_Stains: 0.9815296566077003
vt_score_Stains: 0.9820499479708636
xgb_score_Dirtiness: 0.9771071800208116
rf_score_Dirtiness: 0.977627471383975
vt_score_Dirtiness: 0.9778876170655567
xgb_score_Bumps: 0.7765348595213319
rf_score_Bumps: 0.7749739854318418
vt_score_Bumps: 0.7809573361082206
xgb_score_Other_Faults: 0.6558272632674298
rf_score_Other_Faults: 0.6618106139438086
vt_score_Other_Faults: 0.6683142559833507


In [15]:
# 실제 예측값 생성
xgb_bard = pd.DataFrame({'id':passid})
rf_bard = pd.DataFrame({'id':passid})
vt_bard = pd.DataFrame({'id':passid})
for target_name in target_col:
    y_sub = xgb_model[target_name].predict_proba(test_df)[:, 1]
    xgb_bard[target_name] = y_sub
    
    y_sub = rf_model[target_name].predict_proba(test_df)[:, 1]
    rf_bard[target_name] = y_sub
    
    y_sub = vt_model[target_name].predict_proba(test_df)[:, 1]
    vt_bard[target_name] = y_sub
    
xgb_bard.to_csv('xgb_dev.csv', index=False)
# kaggle score : 0.87257
rf_bard.to_csv('rf_dev.csv', index=False)
# kaggle score : 0.87133
vt_bard.to_csv('vt_dev.csv', index=False)
# kaggle score : 0.87760

In [16]:
import joblib

joblib.dump(rf_model, './rf_dev.pkl')
joblib.dump(xgb_model, './xgb_dev.pkl')
joblib.dump(vt_model, './rf_dev.pkl')

['./rf_dev.pkl']

In [ ]:
xgb_test = pd.DataFrame()
rf_test = pd.DataFrame()
vt_test = pd.DataFrame()

# roc-auc 곡선
for target_name in target_col:
    y_sub = xgb_model[target_name].predict_proba(X_test)[:, 1]
    xgb_test[target_name] = y_sub
    
    y_sub = rf_model[target_name].predict_proba(X_test)[:, 1]
    rf_test[target_name] = y_sub
    
    y_sub = vt_model[target_name].predict_proba(X_test)[:, 1]
    vt_test[target_name] = y_sub

In [ ]:
from sklearn.metrics import roc_auc_score
sum_xgb = 0
sum_rf = 0
sum_vt = 0

for target_name in target_col:
    score_xgb = roc_auc_score(y_test[target_name], xgb_test[target_name], average=None)
    score_rf = roc_auc_score(y_test[target_name], rf_test[target_name], average=None)
    score_vt = roc_auc_score(y_test[target_name], vt_test[target_name], average=None)
    
    sum_xgb += score_xgb
    sum_rf += score_rf
    sum_vt += score_vt
    
sum_xgb = sum_xgb/len(target_col)
sum_rf = sum_rf/len(target_col)
sum_vt = sum_vt/len(target_col)

print("Xgboost:", sum_xgb)
print("Random Forest:", sum_rf)
print("Voting Ensemble:", sum_vt)

## PFI

In [9]:
# pfi 확인 - random forest
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

# target을 분할하여 반복적으로 pfi 계산
for idx in y_test:
    pfi = permutation_importance(rf_model[idx], X_test, y_test[idx], n_repeats = 30, random_state = 42)
    print(f'{idx} :')
    for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
        print(f"{X_test.columns[i] : <8}", "  "
              f"{pfi.importances_mean[i] : .3f}", "  "    
              f"+/- {pfi.importances_std[i] : .3f}")
    print("\n")

Pastry :
Length_of_Conveyer    0.002   +/-  0.001
Orientation_Index    0.002   +/-  0.001
Square_Index    0.002   +/-  0.000
Outside_X_Index    0.002   +/-  0.001
Minimum_of_Luminosity    0.001   +/-  0.001
Edges_Y_Index    0.001   +/-  0.001
Luminosity_Index    0.001   +/-  0.001
LogOfAreas    0.001   +/-  0.000
Steel_Plate_Thickness    0.001   +/-  0.000
Empty_Index    0.001   +/-  0.000
X_Maximum    0.001   +/-  0.000
Sum_of_Luminosity    0.001   +/-  0.000
X_Perimeter    0.001   +/-  0.000
Outside_Global_Index    0.001   +/-  0.001
Edges_Index    0.001   +/-  0.001
X_Minimum    0.001   +/-  0.000
Edges_X_Index    0.001   +/-  0.000
Y_Minimum    0.001   +/-  0.000
Pixels_Areas    0.001   +/-  0.000
Log_Y_Index    0.001   +/-  0.000
TypeOfSteel_A400    0.001   +/-  0.000
Y_Perimeter    0.000   +/-  0.000
TypeOfSteel_A300    0.000   +/-  0.000
Maximum_of_Luminosity    0.000   +/-  0.001
Log_X_Index    0.000   +/-  0.001
Y_Maximum    0.000   +/-  0.000
SigmoidOfAreas    0.000   +/-  0.

In [10]:
# pfi 확인 - xgboost
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

# target을 분할하여 반복적으로 pfi 계산
for idx in y_test:
    pfi = permutation_importance(xgb_model[idx], X_test, y_test[idx], n_repeats = 30, random_state = 42)
    print(f'{idx} :')
    for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
        print(f"{X_test.columns[i] : <8}", "  "
              f"{pfi.importances_mean[i] : .3f}", "  "
              f"+/- {pfi.importances_std[i] : .3f}")
    print("\n")

Pastry :
Minimum_of_Luminosity    0.005   +/-  0.002
SigmoidOfAreas    0.003   +/-  0.001
Sum_of_Luminosity    0.002   +/-  0.001
X_Perimeter    0.002   +/-  0.001
Steel_Plate_Thickness    0.002   +/-  0.001
Y_Maximum    0.002   +/-  0.001
LogOfAreas    0.001   +/-  0.001
Y_Minimum    0.001   +/-  0.001
Pixels_Areas    0.001   +/-  0.001
Maximum_of_Luminosity    0.001   +/-  0.001
Luminosity_Index    0.001   +/-  0.001
Square_Index    0.001   +/-  0.001
Log_Y_Index    0.001   +/-  0.001
X_Minimum    0.001   +/-  0.001
Orientation_Index    0.001   +/-  0.001
Outside_X_Index    0.001   +/-  0.001
TypeOfSteel_A300    0.000   +/-  0.001
Edges_Y_Index    0.000   +/-  0.001
Length_of_Conveyer    0.000   +/-  0.001
Log_X_Index    0.000   +/-  0.001
Edges_Index    0.000   +/-  0.001
Y_Perimeter   -0.000   +/-  0.001
TypeOfSteel_A400   -0.000   +/-  0.000
Outside_Global_Index   -0.001   +/-  0.001
Edges_X_Index   -0.001   +/-  0.001
X_Maximum   -0.001   +/-  0.001
Empty_Index   -0.001   +/-  0.

In [11]:
# pfi 확인 - Voting Classifer
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

# target을 분할하여 반복적으로 pfi 계산
for idx in y_test:
    pfi = permutation_importance(vt_model[idx], X_test, y_test[idx], n_repeats = 30, random_state = 42)
    print(f'{idx} :')
    for i in pfi.importances_mean.argsort()[::-1]: #argsort = 인덱스 정렬
        print(f"{X_test.columns[i] : <8}", "  "
              f"{pfi.importances_mean[i] : .3f}", "  "    
              f"+/- {pfi.importances_std[i] : .3f}")
    print("\n")

Pastry :
Minimum_of_Luminosity    0.003   +/-  0.001
Sum_of_Luminosity    0.002   +/-  0.001
SigmoidOfAreas    0.002   +/-  0.001
Orientation_Index    0.001   +/-  0.001
Length_of_Conveyer    0.001   +/-  0.001
Steel_Plate_Thickness    0.001   +/-  0.001
Edges_Y_Index    0.001   +/-  0.001
Outside_X_Index    0.001   +/-  0.001
Luminosity_Index    0.001   +/-  0.001
Maximum_of_Luminosity    0.000   +/-  0.001
Square_Index    0.000   +/-  0.001
Log_X_Index    0.000   +/-  0.001
Y_Perimeter    0.000   +/-  0.001
X_Perimeter    0.000   +/-  0.001
Outside_Global_Index    0.000   +/-  0.001
Pixels_Areas    0.000   +/-  0.001
X_Minimum    0.000   +/-  0.001
Edges_X_Index   -0.000   +/-  0.001
TypeOfSteel_A300   -0.000   +/-  0.001
TypeOfSteel_A400   -0.000   +/-  0.000
Empty_Index   -0.000   +/-  0.001
X_Maximum   -0.000   +/-  0.001
Log_Y_Index   -0.000   +/-  0.001
Y_Maximum   -0.001   +/-  0.001
Edges_Index   -0.001   +/-  0.001
Y_Minimum   -0.001   +/-  0.001
LogOfAreas   -0.001   +/-  0.

In [61]:
train_col = train_df.columns

In [48]:
for i in train_col:
    print(i)

Pixels_Areas
X_Perimeter
Y_Perimeter
Sum_of_Luminosity
Length_of_Conveyer
TypeOfSteel_A300
TypeOfSteel_A400
Steel_Plate_Thickness
Edges_Index
Empty_Index
Square_Index
Outside_X_Index
Edges_X_Index
Edges_Y_Index
Outside_Global_Index
LogOfAreas
Log_X_Index
Log_Y_Index
Orientation_Index
Luminosity_Index
SigmoidOfAreas
X_range
Y_range
Luminosity_Range


# 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.metrics import roc_curve
# xgboost
for target in y_train:
    xgb = train_xgboost(y_train[target])
    pred_xgb, y_xgb = prediction_xgboost(y_train[target],xgb)
    pred = pd.DataFrame(y_xgb)
    pred.rename(columns = {0:target}, inplace = True)
    sub_xgb = pd.concat([sub_xgb, pred], axis = 1)

In [ ]:
# optuna를 이용한 xgboost 하이퍼 파라미터 튜닝
study = optuna.create_study()
study.optimize(opt_bumps, n_trials = 50, timeout = 3600)
print(study.best_params)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_df, target, random_state= 42, test_size=0.2)

best_xgb = xgboost.XGBClassifier(**study.best_params)
best_xgb.fit(X_train, y_train)
y_pred = best_xgb.predict_proba(test_df)

In [ ]:
submission = pd.DataFrame(y_pred)

for col, idx in enumerate(target.columns):
    tar = {col : idx}
    submission = submission.rename(tar, axis = 1)
    
submission.insert(0,'id', sample['id'])
submission.to_csv('./submission_xgboost_opt.csv',index= False)

In [ ]:
from sklearn.model_selection import train_test_split

train_pre = pd.concat([train_df, target], axis = 1)

feature_train = train_pre.columns.to_list()
feature_test = test_df.columns.to_list()
target_list = target.columns.to_list()

X_train, X_test, y_train, y_test = train_test_split(train_pre, target, test_size=0.2)